In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd gdrive/My Drive/Colab

/content/gdrive/My Drive/Colab


In [0]:
ls

fer2013.csv  hello/  PrivateTest/  PublicTest/  Training/


In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.decomposition import PCA
import seaborn as sns

from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2 as cv
import time
import copy as cp
import pandas as pd
import pickle 

import torch
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [0]:
def load1(path_train,path_test1,path_test2):
    transform = transforms.Compose([transforms.Resize(227),transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    batch_size=32
    
    trainloader = torch.utils.data.DataLoader(
          ImageFilelist(root=path_train,lbl='Training', 
          transform=transform),
          batch_size=batch_size, shuffle=False, pin_memory=True)
    
    testloader1 = torch.utils.data.DataLoader(
          ImageFilelist(root=path_test1,lbl='PublicTest',
          transform=transform),
          batch_size=batch_size, shuffle=False, pin_memory=True)

    testloader2 = torch.utils.data.DataLoader(
          ImageFilelist(root=path_test2,lbl='PrivateTest',
          transform=transform),
          batch_size=batch_size, shuffle=False, pin_memory=True)

          
    return trainloader,testloader1,testloader2


In [0]:
def train(trainloader,net):
    trn=[]; lbl=[]; c=0; t=time.time()
    with torch.no_grad():
         for data1 in trainloader:
             c+=1
             if(c%100==0):
                 print('Training: ',c,', Time: ',time.time()-t)
                 t=time.time()
             images, labels = data1
             images, labels = torch.tensor(images), torch.tensor(labels)
             images, labels = images.to(device), labels.to(device)
             out = net(images)
             for outputs in out:
                trn.append(cp.deepcopy(np.array(outputs.cpu(),np.double)))
             for label in labels:
                lbl.append(cp.deepcopy(np.array(label.cpu(),np.int32)))
    return np.array(trn),np.array(lbl)
         
    
def test(testloader,net):
    tst=[]; lbl=[]; c=0; t=time.time()
    with torch.no_grad():
         for data1 in testloader:
             c+=1
             if(c%50==0):
                 print('Testing: ',c,', Time: ',time.time()-t)
                 t=time.time()
             images, labels = data1
             images, labels = torch.tensor(images), torch.tensor(labels)
             images, labels = images.to(device), labels.to(device)
             out = net(images)
             for outputs in out:  
                tst.append(cp.deepcopy(np.array(outputs.cpu(),np.double)))
             for label in labels:
               lbl.append(cp.deepcopy(np.array(label.cpu(),np.int32)))
    return np.array(tst),np.array(lbl)

In [0]:
 def default_flist_reader(flist,lbl):
    imlist=[]
    path='fer2013.csv'
    data=pd.read_csv(path)
    emotion={0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Sad', 5:'Surprise', 6:'Neutral'}
    for i in range(len(data.pixels)):
        if(lbl==data.Usage[i]):
          im=np.array(data.pixels[i].split(' '),np.uint8)
          im.shape=(48,48)
          im=Image.fromarray(im).convert('RGB')
          nm=data.emotion[i]
          imlist.append(cp.deepcopy((im,nm)))    
    print(len(imlist))
    return imlist
   
class ImageFilelist(data.Dataset):
     def __init__(self, root, transform=None,lbl='Training', target_transform=None,
         flist_reader=default_flist_reader):
         self.lbl=lbl
         self.imlist = flist_reader(root,lbl)
         self.transform = transform
         self.target_transform = target_transform
         
   
     def __getitem__(self, index):
         img, target = self.imlist[index]
         #img = self.loader(impath)
         if self.transform is not None:
            img = self.transform(img)
         if self.target_transform is not None:
            target = self.target_transform(target)
         return img, target
   
     def __len__(self):
         return len(self.imlist)


**Load Data**

In [0]:
p_trn='/Training/'
p_tst1='/PublicTest/'
p_tst2='/PrivateTest/'
trainloader,testloader1,testloader2=load1(p_trn,p_tst1,p_tst2)

FileNotFoundError: ignored

In [0]:
alexnet = torchvision.models.alexnet(True)
#vgg = torchvision.models.vgg16(True)
#net=torchvision.models.densenet161(True)
#alexnet.to(device)
#vgg.to(device)
print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.torch/models/alexnet-owt-4df8aa71.pth
244418560it [00:02, 107756163.98it/s]


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [0]:
net1.classifier=nn.Sequential(*list(alexnet.classifier.children())[:-5])
net2.classifier=nn.Sequential(*list(vgg.classifier.children())[:-3])

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
  )
)


**Extract** **Features**

In [0]:
Xtr1,Ytr1=train(trainloader,net1)
print(Xtr1.shape,Ytr1.shape)
Xtr2,Ytr2=train(trainloader,net2)
print(Xtr2.shape,Ytr2.shape)


Training:  100 , Time:  11.387022256851196
Training:  200 , Time:  11.457809209823608
Training:  300 , Time:  11.744714975357056
Training:  400 , Time:  11.202537536621094
Training:  500 , Time:  11.148491621017456
Training:  600 , Time:  11.10200309753418
Training:  700 , Time:  11.719087600708008
Training:  800 , Time:  11.74514889717102
(28709, 4096) (28709,)
Training:  100 , Time:  29.98655652999878
Training:  200 , Time:  31.138575792312622
Training:  300 , Time:  30.935880184173584
Training:  400 , Time:  30.838395833969116
Training:  500 , Time:  30.800657510757446
Training:  600 , Time:  31.01133894920349
Training:  700 , Time:  31.134522199630737
Training:  800 , Time:  31.333293914794922
(28709, 4096) (28709,)


In [0]:
Xte1_pub,Yte1_pub=test(testloader1,net1)
print(Xte1_pub.shape,Yte1_pub.shape)

Xte2_pub,Yte2_pub=test(testloader1,net2)
print(Xte2_pub.shape,Yte2_pub.shape)

Testing:  50 , Time:  5.876577377319336
Testing:  100 , Time:  5.439253091812134
(3589, 4096) (3589,)
Testing:  50 , Time:  15.146250009536743
Testing:  100 , Time:  15.654850959777832
(3589, 4096) (3589,)


In [0]:
Xte1_pri,Yte1_pri=test(testloader2,net1)
print(Xte1_pri.shape,Yte1_pri.shape)

Xte2_pri,Yte2_pri=test(testloader2,net2)
print(Xte2_pri.shape,Yte2_pri.shape)

Testing:  50 , Time:  5.285412788391113
Testing:  100 , Time:  5.959738254547119
(3589, 4096) (3589,)
Testing:  50 , Time:  15.044383764266968
Testing:  100 , Time:  15.33812928199768
(3589, 4096) (3589,)


In [0]:
Xtr=[];
for i in range(Ytr1.shape[0]):
  tmp=np.concatenate((Xtr1[i],Xtr2[i]))
  Xtr.append(cp.deepcopy(tmp))
Xtr=np.array(Xtr)
print(Xtr.shape)

Xte_pub=[];
for i in range(Yte1_pub.shape[0]):
  tmp=np.concatenate((Xte1_pub[i],Xte2_pub[i]))
  Xte_pub.append(cp.deepcopy(tmp))
Xte_pub=np.array(Xte_pub)
print(Xte_pub.shape)
  
Xte_pri=[];
for i in range(Yte1_pri.shape[0]):
  tmp=np.concatenate((Xte1_pri[i],Xte2_pri[i]))
  Xte_pri.append(cp.deepcopy(tmp))
Xte_pri=np.array(Xte_pri)
print(Xte_pri.shape)

(28709, 8192)
(3589, 8192)
(3589, 8192)


In [0]:
np.unique(Ytr1)

array([0, 1, 2, 3, 4, 5, 6], dtype=int32)

**Preprocessing**

In [0]:
Xtr1=preprocessing.scale(Xtr)
Xte1_pub=preprocessing.scale(Xte_pub)
Xte1_pri=preprocessing.scale(Xte_pri)

In [0]:
pca=PCA(n_components=1024)
pca.fit(Xtr)
print(pca.explained_variance_ratio_.sum())

0.9303377095653136


In [0]:
Xtr1=pca.transform(Xtr)
Xte1_pub=pca.transform(Xte_pub)
Xte1_pri=pca.transform(Xte_pri)
print(Xtr1.shape,Xte1_pub.shape,Xte1_pri.shape)

(28709, 1024) (3589, 1024) (3589, 1024)


**Classification**


In [0]:
#clf=LinearSVC()
clf=SVC(kernel='rbf',probability=True)
clf.fit(Xtr1,Ytr1)

**------------------------------------------------------------------ Public Data ----------------------------------------------------------------------------**

In [0]:
p_lbl=clf.predict(Xte1_pub)
cmat=confusion_matrix(Yte1_pub,p_lbl)
sns.heatmap(cmat,annot=True,fmt='g')
print(clf.score(Xte1_pub,Yte1_pub))

ROC Plot

In [0]:
prob=clf.predict_proba(Xte1_pub)
scores=[]
for i in range(7):
    fpr, tpr, thresholds = metrics.roc_curve(Yte1_pub, prob[:,i], pos_label=i)
    plt.plot(fpr,tpr,label='C'+str(i))
    plt.legend()
plt.xlabel('FPR')
plt.ylabel('TPR')  
plt.title(' Receiver Operating Characteristic Curve of Public Data')

**------------------------------------------------------------------ Private Data ----------------------------------------------------------------------------**

In [0]:
p_lbl=clf.predict(Xte1_pri)
cmat=confusion_matrix(Yte1_pri,p_lbl)
sns.heatmap(cmat,annot=True,fmt='g')
print(clf.score(Xte1_pri,Yte1_pri))

ROC Plot

In [0]:
prob=clf.predict_proba(Xte1_pri)
scores=[]
for i in range(7):
    fpr, tpr, thresholds = metrics.roc_curve(Yte1_pri, prob[:,i], pos_label=i)
    plt.plot(fpr,tpr,label='C'+str(i))
    plt.legend()
plt.xlabel('FPR')
plt.ylabel('TPR')  
plt.title('Receiver Operating Characteristic Curve of Private Data')

****

**Save** **Features**


In [0]:
ls

PrivateTest_feat.pickle  PublicTest_feat.pickle  Training_feat.pickle
PrivateTest_lbl.pickle   PublicTest_lbl.pickle   Training_lbl.pickle


In [0]:
with open('Training_feat.pickle', 'wb') as handle1:
    pickle.dump(Xtr, handle1)

In [0]:
with open('Training_lbl.pickle', 'wb') as handle2:
    pickle.dump(Ytr, handle2)

In [0]:
with open('PublicTest_feat.pickle', 'wb') as handle3:
    pickle.dump(Xte_pub, handle3)

In [0]:
with open('PublicTest_lbl.pickle', 'wb') as handle4:
    pickle.dump(Yte_pub, handle4)

In [0]:
with open('PrivateTest_feat.pickle', 'wb') as handle5:
    pickle.dump(Xte_pri, handle5)

In [0]:
with open('PrivateTest_lbl.pickle', 'wb') as handle6:
    pickle.dump(Yte_pri, handle6)

**Misc work**

In [0]:
import zipfile

In [0]:
!unzip testData.zip

In [0]:
with ZipFile('testData.zip', 'r') as zipObj:
     zipObj.extractall()

TypeError: ignored

**References**

In [0]:
[1] https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
[2] https://pytorch.org/docs/stable/torchvision/models.html
[3] https://pytorch.org/docs/stable/data.html
[4] https://github.com/pytorch/vision/issues/81